In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed
import pandas as pd
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

plt.rcParams["figure.figsize"] = [12, 9]

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


## Announcements

  - Quiz 3 in recitation this week. 
    - Curves, tangents
    - Motion
    - Arc length
  - Homework 4 posted, due 10/01
  - CA Help Room (New Rooms!) open 
    - Thur 7–9pm - ET 253
    - Sun 3–5pm - Mudd 337
    - Mon 7–9pm - ET 253
    

# One-minute Review

A **scalar field** (e.g., $f(x,y)$) is a function of several variables. Its **domain** is the subset of input values in $\RR^n$; its **image** is the set of output values in $\RR$.

**Level sets** ("curves" for functions of 2 variables) are sets of input points associated to a particular output. For example, contour lines on a topographical map.

![Map of Adirondack region](https://www.nyroute28.com/maps/oldforgetopo.jpg)

## Level Curves

In [28]:
f = lambda x,y: x*y
g = lambda x,y: x*sin(y)
h = lambda x,y: sqrt(4-x**2-y**2)
k = lambda x,y: log(x**2+y**2)
l = lambda x,y: exp((x-y)*log(2))


@interact
def _(lev=(0.01,1.),prob={"1":[f,r"$z = x y$"],"2":[g,r"$z = x \sin(y)$"],"3":[h,r"$z = \sqrt{4-x^2-y^2}$"],"4":[k,r"$z = \ln (x^2+y^2)$"],"5":[l,r"$z = 2^{x-y}$"]},angle=(-90,120,6),vangle=(0,90,6)):
    func,fs = prob
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(121,projection='3d')
    ax.view_init(vangle,angle)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    x = y = np.linspace(-1,1,400)
    X,Y = np.meshgrid(x,y)
    if func == h:
        x = np.linspace(0,2*pi,100)
        y = np.linspace(0,1.99,100)
        x,y = np.meshgrid(x,y)
        X = y*cos(x)
        Y = y*sin(x)
        ax.set_zlim3d([0,4])
    else:
        X = 5*X
        Y = 5*Y
    Z = func(X,Y)
#     ax.set_autoscale_on(True)
    ax.plot_surface(X,Y,Z,alpha=.3,cmap='viridis',rcount=75,ccount=75);
    k = np.max(Z)*(lev)+(1-lev)*np.min(Z)
    ax.contour(X,Y,Z,offset=k,levels=[k],colors=['red'])
    ax.set_title(fs)
    ax2 = fig.add_subplot(122)
    cp = ax2.contour(X,Y,Z,cmap='viridis');
    # fig.colorbar(cp); # for colorbar reference
    ax2.clabel(cp,fmt='%1.1f'); # inline counour labels.
    cp2 = ax2.contour(X,Y,Z,levels=[k],colors=['red'])
    ax2.clabel(cp2,fmt='%1.1f'); # inline counour labels.

interactive(children=(FloatSlider(value=0.505, description='lev', max=1.0, min=0.01), Dropdown(description='pr…

# Lecture 08

  - Objectives

    - Explore limits and continuity of $f(x,y)$.
    - Define partial derivatives
    - Estimate partial derivatives from contour maps and tables.
  
  - Resources
    - Content
      - Stewart: §14.2—3
      - New Strang: [§4.2](https://cnx.org/contents/oxzXkyFi@5.30:2YObsFkq@7/4-2-Limits-and-Continuity)
      - [Slides](https://hub.columbiajupyter2.org/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdrewyoungren%2Fmvc-f19&urlpath=tree%2Fmvc-f19%2Fslides%2Fmvc-L08.ipynb) via JupyterHub
    - Visualization
      - [CalcPlot3D](https://www.monroecc.edu/faculty/paulseeburger/calcnsf/CalcPlot3D/)
    - Practice
      - Mooculus: [Continuity](https://ximera.osu.edu/mooculus/calculus3/continuityOfFunctionsOfSeveralVariables/digInContinuity) [Partial Derivatives](https://ximera.osu.edu/mooculus/calculus3/partialDerivativesAndTheGradientVector/digInPartialDerivatives)
    - Extras
      - CalcBLUE: [Partial Derivatives](https://youtu.be/3QqfUIbQpfg)

# Limits

Consider a function $f:\RR^n \to \RR$ as mapping vectors to scalars. We write $$\lim_{\vec x \to \vec p} f(\vec x) = L$$ if $|f(\vec x) - L|$ can be made arbitrarily small by making $|\vec x - \vec p|$ sufficiently small. 

# Limits in $\RR^2$

Consider a function $f:\RR^2 \to \RR$ as mapping vectors to scalars. We write $$\lim_{(x,y) \to (a,b)} f(x,y) = L$$ if $|f(x,y) - L|$ can be made arbitrarily small by making $\sqrt{(x-a)^2+(y-b)^2}$ sufficiently small. 

### Examples

See [This screencast](https://youtu.be/EQUBHl3X7oU) for a few more/more detail. 

In [33]:
f = lambda x,y: (x**2 + y**2)
g = lambda x,y: x*y/(x**2 + y**2)
h = lambda x,y: x*y/sqrt(x**2 + y**2)


@interact
def _(lev=(0.01,1.),prob={"1":[f,r"$z = x^2 + y^2$"],"2":[g,r"$z = \frac{xy}{x^2 + y^2}$"],"3":[h,r"$z = \frac{xy}{\sqrt{x^2+ y^2}}$"]},angle=(-90,120,6),vangle=(0,90,6)):
    func,fs = prob
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(121,projection='3d')
    ax.view_init(vangle,angle)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    x = y = np.linspace(-1,1,400)
    X,Y = np.meshgrid(x,y)
    Z = func(X,Y)
#     ax.set_autoscale_on(True)
    ax.plot_surface(X,Y,Z,alpha=.7,cmap='viridis',rcount=75,ccount=75);
    k = np.max(Z)*(lev)+(1-lev)*np.min(Z)
    ax.contour(X,Y,Z,offset=k,levels=[k],colors=['red'])
    ax.set_title(fs)
    ax2 = fig.add_subplot(122)
    cp = ax2.contour(X,Y,Z,cmap='viridis');
    # fig.colorbar(cp); # for colorbar reference
    ax2.clabel(cp,fmt='%1.1f'); # inline counour labels.
    cp2 = ax2.contour(X,Y,Z,levels=[k],colors=['red'])
    ax2.clabel(cp2,fmt='%1.1f'); # inline counour labels.

interactive(children=(FloatSlider(value=0.505, description='lev', max=1.0, min=0.01), Dropdown(description='pr…

# Rates of Change

Limits went well. Let's try derivatives as a limit of a difference quotient. 

$$ \lim_{\langle x,y\rangle \to \langle a,b \rangle} \frac{f(x,y) - f(a,b)}{\langle x,y\rangle - \langle a,b \rangle}$$

# Blech!

**One cannot divide by vectors!**

## A sensible question

A group of hikers follows a curving path up a mountain ridge. How steep is their path at the halfway point?


In [57]:
f = lambda x,y: exp(-4*(y-sin(x))**2)*(1-np.abs(x+pi/2)/5)



@interact
def _(func=fixed(f),angle=(-90,120,6),vangle=(0,90,6)):
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(111,projection='3d')
    ax.view_init(vangle,angle)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    x =  np.linspace(-4,4,601)
    y =  np.linspace(-2,2,301)    
    X,Y = np.meshgrid(x,y)
    Z = func(X,Y)
    ax.plot_surface(X,Y,Z,alpha=.6,cmap='ocean',rcount=100,ccount=100);
    t = np.linspace(-pi/2,pi/2,100)
    X = t
    Y = t/pi - 1/2 - t*(t**2-pi**2/4)/5
    Z = func(X,Y)
    ax.plot(X,Y,Z,lw=6,color='r',alpha=1)

interactive(children=(IntSlider(value=12, description='angle', max=120, min=-90, step=6), IntSlider(value=42, …

# Partial Derivatives

We start by considering "one direction at a time".

The **partial derivative** of a function $f(x,y)$ with respect to $x$ at the point $(a,b)$ is $$f_x(a,b) = \lim_{h\to 0} \frac{f(a+h,b) - f(a,b)}{h}.$$


The **partial derivative** of a function $f(x,y)$ with respect to $y$ at the point $(a,b)$ is $$f_y(a,b) = \lim_{h\to 0} \frac{f(a,b+h) - f(a,b)}{h}.$$

In [60]:
f = lambda x,y: exp(-4*(y-sin(x))**2)*(1-np.abs(x+pi/2)/5)



@interact
def _(func=fixed(f),angle=(-90,120,6),vangle=(0,90,6),var=['x','y']):
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(111,projection='3d')
    ax.view_init(vangle,angle)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    x =  np.linspace(-4,4,601)
    y =  np.linspace(-2,2,301)    
    X,Y = np.meshgrid(x,y)
    Z = func(X,Y)
    ax.plot_surface(X,Y,Z,alpha=.6,cmap='ocean',rcount=100,ccount=100);
    t = np.linspace(-pi/2,pi/2,100)
    X = t
    Y = np.zeros_like(X)
    if var == 'y':
        X,Y = Y,X
    Z = func(X,Y)
    ax.plot(X,Y,Z,lw=6,color='r',alpha=1)

interactive(children=(IntSlider(value=12, description='angle', max=120, min=-90, step=6), IntSlider(value=42, …


##### Other notation

All of these are equivalent. 

$$f_x = \frac{\partial f}{\partial x} = \partial_x f = f^{(1,0)}$$

and there are many more. 

### Computing $\frac{\partial f}{\partial x}$

In practice, we compute partial derivatives by **treating all variables except the variable in question as constant**.

#### Example

Compute:

  1. $\displaystyle \frac{\partial}{\partial x} \left( x^2y - \sin(x-2y) \right)$

  2. $\displaystyle \frac{\partial}{\partial y} \left( x^2y - \sin(x-2y) \right)$

  3. $\displaystyle \frac{\partial}{\partial z} \left( \frac{z^2 \tan^{-1}(\sqrt{x^2+1})}{\cosh(xy)} \right)$